In [1]:
from twitter_reader.twitter_manager.component import APIConfig, query
from twitter_reader.data_storage.component import save_tweets, DBConfig, read_tweets, clear_tweet_table

In [2]:
tm_conf = APIConfig()
tweets = query('nike', tm_conf)

got 10 tweets.


In [3]:
db_conf = DBConfig()

In [4]:
clear_tweet_table(db_conf)
save_tweets(response['data'], db_conf)

TypeError: list indices must be integers or slices, not str

In [ ]:
tweets = read_tweets(db_conf)

In [ ]:
len(tweets)